In [194]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, validation_curve, train_test_split
from sklearn.preprocessing import StandardScaler

In [195]:
df = pd.read_csv("DATA/train.csv")
df = df.drop(columns=['Unnamed: 0'])

In [196]:
features = ['DAYOFMONTH', 'CRSDEPTIME', 'DISTANCEGROUP', 'DEPDEL15']

In [197]:
X = df[features]
y = df['DEPDEL15']
X = X.drop(columns=['DEPDEL15'])

In [198]:
X

,DAYOFMONTH,CRSDEPTIME,DISTANCEGROUP
0,11,1452,2
1,12,2015,2
2,11,1118,4
3,27,1600,3
4,4,710,1
...,...,...,...
386486,9,600,3
386487,3,1635,3
386488,29,1506,3
386489,17,540,8


In [199]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

In [200]:
scaler = StandardScaler()
scaler.fit(x_train)  
x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)  

In [201]:
clf = SGDClassifier()
clf.fit(x_train, y_train)
clf.score(x_val, y_val)

0.7762014006278677

In [202]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)
clf.score(x_val, y_val)

0.727981508952289

In [203]:
from sklearn.ensemble import RandomForestClassifier

In [204]:
# clf = RandomForestClassifier()
# clf.fit(x_train, y_train)
# clf.score(x_val, y_val)

In [205]:
from xgboost import XGBClassifier

In [206]:
xg_reg = XGBClassifier(enable_categorical=True)
xg_reg.fit(x_train, y_train)
xg_reg.score(x_val, y_val)

0.7769344878738745

In [207]:
test = pd.read_csv('DATA/test_data.csv')

In [208]:
test.columns = test.columns.str.upper()
test['FLIGHTDATE'] = pd.to_datetime(test['FLIGHTDATE'])
test['CRSDEPTIME'] = test['CRSDEPTIME'].astype(str)
test['CRSDEPTIME'] = test['CRSDEPTIME'].str.zfill(4)
test['DEPHOUR'] = test['CRSDEPTIME'].str[:-2]
test['DEPMINUTE'] = test['CRSDEPTIME'].str[-2:]
test['DEPHOUR'] = test['DEPHOUR'].str.zfill(2)
test['DEPMINUTE'] = test['DEPMINUTE'].str.zfill(2)

flight_counts = test.groupby(['ORIGIN', 'MONTH', 'DAYOFMONTH', 'DEPHOUR']).size().reset_index(name='FLIGHTSSAMETIME')
test = test.merge(flight_counts, on=['ORIGIN', 'MONTH', 'DAYOFMONTH', 'DEPHOUR'], how='left')
test['FLIGHTSSAMETIME'] = test['FLIGHTSSAMETIME'].fillna(0).astype(int)

In [209]:
test = test[features]

In [210]:
y_true = test['DEPDEL15']
test = test.drop(columns='DEPDEL15')

In [214]:
test = scaler.transform(test)

In [215]:
test_pred = xg_reg.predict(test)

In [216]:
accuracy_score(y_true, test_pred)

0.771931731053677